In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
cwd = os.getcwd()
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [3]:
from typing import Any

import dill
from sklearn.linear_model import LogisticRegression
import numpy as np

from CrossValidation import cross_validation
from MIRA import CostSensitiveMIRA
from Settings import Settings

from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model_parallel, train_model, train_cost_sensitive_instance
from window_based_tagger_config import get_config
from searn_parser_breadth_first import SearnModelBreadthFirst
from causal_model_features import CausalModelType

In [4]:
from parser_inputs import ParserInputs, ParserInputsEssayLevel

In [5]:
from feature_extraction import get_features_from_probabilities

In [6]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
CAUSAL_MODEL_TYPE = CausalModelType.CORAL_BLEACHING
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

config = get_config(training_folder)

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
902 226


In [7]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:6->Result:7',
 'Causer:12->Result:11',
 'Causer:11->Result:11',
 'Causer:3->Result:13',
 'Causer:11->Result:6',
 'Causer:5->Result:4',
 'Causer:1->Result:6',
 'Causer:12->Result:50',
 'Causer:3->Result:5b',
 'Causer:50->Result:3']

In [8]:
base_extractors, all_extractor_fns, all_cost_functions = create_extractor_functions()

all_extractor_fn_names = get_function_names(all_extractor_fns)
base_extractor_fn_names = get_function_names(base_extractors)
all_cost_fn_names = get_function_names(all_cost_functions)

ngrams = 1
stemmed = True
cost_function_name = micro_f1_cost_plusepsilon.__name__
dual = True
fit_intercept = True
beta = 0.5
max_epochs = 2
lr_C = 0.5
penalty = "l2"

In [9]:
BASE_LEARNER_FACT = lambda: LogisticRegression(dual=dual, C=lr_C, penalty=penalty, fit_intercept=fit_intercept)
best_extractor_names = ['single_words', 'between_word_features', 'label_set',
                        'three_words', 'third_order', 'unigrams']  # type: List[str]

test_folds = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]
cv_folds = cross_validation(pred_tagged_essays_train, CV_FOLDS)  # type: List[Tuple[Any,Any]]

result_test_essay_level = evaluate_model_essay_level(
    folds=cv_folds,
    extractor_fn_names_lst=best_extractor_names,
    all_extractor_fns=all_extractor_fns,
    ngrams=ngrams,
    beta=beta,
    stemmed=stemmed,
    max_epochs=max_epochs, 
    min_feat_freq=MIN_FEAT_FREQ, 
    cr_tags=set_cr_tags,
    base_learner_fact=BASE_LEARNER_FACT, 
    down_sample_rate=1.0, model = SearnModelBreadthFirst)

## Training Accuracy

In [10]:
models, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_td_preds_by_sent, cv_sent_vd_ys_by_tag = result_test_essay_level

mean_metrics = ResultsProcessor.compute_mean_metrics(cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag)
get_micro_metrics(metrics_to_df(mean_metrics))

,accuracy,f1_score,recall,precision
95,0.985734,0.780794,0.761131,0.801499


## CV Accuracy

In [11]:
models, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = result_test_essay_level

mean_metrics = ResultsProcessor.compute_mean_metrics(cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag)
get_micro_metrics(metrics_to_df(mean_metrics))

,accuracy,f1_score,recall,precision
95,0.982895,0.740481,0.731022,0.750187


## Test Accuracy

In [12]:
result_final_test = evaluate_model_essay_level(
    folds=test_folds,
    extractor_fn_names_lst=best_extractor_names,
    all_extractor_fns=all_extractor_fns,
    ngrams=ngrams,
    beta=beta,
    stemmed=stemmed,
    max_epochs=max_epochs,
    min_feat_freq=MIN_FEAT_FREQ, 
    cr_tags=set_cr_tags,
    base_learner_fact=BASE_LEARNER_FACT, 
    down_sample_rate=1.0, model = SearnModelBreadthFirst)

models_test, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = result_final_test

mean_metrics = ResultsProcessor.compute_mean_metrics(cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag)
get_micro_metrics(metrics_to_df(mean_metrics))

,accuracy,f1_score,recall,precision
95,0.98551,0.739054,0.771481,0.709244


In [13]:
final_test_model = models_test

In [14]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

### Best Parameter Settings

In [15]:
# initial settings for other params
best_top_n, best_C, best_max_upd, best_max_parses, best_min_prob = (2, 0.0005, 2, 300, 0.0)  # min prob of 0 seems better

## Train Re-Ranker

In [16]:
xs_rerank = essay_to_crels_cv(cv_folds, models, top_n=best_top_n, search_mode_max_prob=False)
xs = get_features_from_probabilities(xs_rerank, name2crels, best_max_parses, 
                                     causal_model_type=CAUSAL_MODEL_TYPE,
                                     min_feat_freq=1, min_prob=best_min_prob)

cv_folds_rerank = cross_validation(xs, 5)
cv_folds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_folds_rerank]

In [ ]:
%%time
f1 = train_model_parallel(
    cv_folds=cv_folds_mm, name2essay=name2essay, C=best_C, pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
    set_cr_tags=set_cr_tags, initial_weight=0.01)
print(f1)  # 0.7421167703055035

## Train on Test Data

In [18]:
xs_test_rerank = essay_to_crels_cv(test_folds, final_test_model, top_n=best_top_n, search_mode_max_prob=False)
xs_test = get_features_from_probabilities(xs_test_rerank, name2crels, best_max_parses, 
                                          causal_model_type=CAUSAL_MODEL_TYPE,
                                          min_feat_freq=1, min_prob=best_min_prob)

In [19]:
# training data comes from the test fold predictions from CV on the full training dataset
xs_train = []
for train, test in cv_folds_rerank:
    xs_train.extend(test)

In [20]:
# Normalize both using training data
xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train,xs_test)

In [21]:
num_train = int(0.8 * len(xs_train_mm))
tmp_train_copy = list(xs_train_mm)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [22]:
C = best_C/5 # This needs to be a lot lower
pa_type = 1
loss_type= "ml"
max_update_items = best_max_upd
initial_weight = 0.01 # was 0.01

## Determine Number of Training Iterations

In [23]:
%%time
# use training data to determine number of iterations
mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=5, train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7386 Test Accuracy: 0.7535
Epoch: 1 Train Accuracy: 0.7416 Test Accuracy: 0.7512
Epoch: 2 Train Accuracy: 0.7406 Test Accuracy: 0.7560
Epoch: 3 Train Accuracy: 0.7416 Test Accuracy: 0.7548
Epoch: 4 Train Accuracy: 0.7422 Test Accuracy: 0.7510
Epoch: 5 Train Accuracy: 0.7429 Test Accuracy: 0.7497
Epoch: 6 Train Accuracy: 0.7426 Test Accuracy: 0.7472
Epoch: 7 Train Accuracy: 0.7423 Test Accuracy: 0.7472
Best Test Acc: 0.7560
CPU times: user 2min 50s, sys: 173 ms, total: 2min 50s
Wall time: 2min 50s


In [24]:
best_iterations, best_C, initial_weight

(3, 0.0005, 0.01)

In [25]:
# Test on test data
mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=initial_weight)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  xs_train=xs_train_mm, xs_test=xs_test_mm,
                                       name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=best_iterations, early_stop_iters=best_iterations, train_instance_fn = train_cost_sensitive_instance, verbose=True)
# Best Test Acc: 0.7406

Epoch: 0 Train Accuracy: 0.7423 Test Accuracy: 0.7398
Epoch: 1 Train Accuracy: 0.7435 Test Accuracy: 0.7433
Epoch: 2 Train Accuracy: 0.7439 Test Accuracy: 0.7451
Best Test Acc: 0.7451


# Feature Selection

In [26]:
from filter_features import filter_feats

prefixes = [
    "Prob-",
    "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_",
    "Propn_",
    "Diff_"
]
# xs_fltr_train, xs_fltr_test = filter_feats(xs_train_mm, xs_test_mm, prefixes)
assert len(prefixes) == len(set(prefixes)), "Duplicate prefixes found"

In [27]:
best_f1 = -1
current_best = []
remaining = list(prefixes)

while True:
    if len(remaining) == 0:
        break
    
    f1_by_prefix = dict()
    for prefix in remaining:
        new_prefixes = current_best + [prefix]
        
        cv_filtered = []
        for tr, test in cv_folds_mm:
            x_tr,x_test = filter_feats(tr, test, new_prefixes)
            cv_filtered.append((x_tr,x_test))
        
        f1_by_prefix[prefix] = train_model_parallel(cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
                                  pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
                                  set_cr_tags=set_cr_tags, initial_weight=initial_weight)
    
    best_prefix, new_best_f1 = sorted(f1_by_prefix.items(), key = lambda tpl: -tpl[1])[0]
    if new_best_f1 > best_f1:
        best_f1 = new_best_f1
        current_best.append(best_prefix)
        remaining.remove(best_prefix)
        print("{length} feats, new Best F1: {f1:.4f} Prefixes: {prefixes}".format(
            length=len(current_best), f1=best_f1, prefixes=str(current_best)))
    else:
        print("No further improvement, stopping")
        break
        
# 1 feats, new Best F1: 0.7389 Prefixes: ['CREL_']
# 2 feats, new Best F1: 0.7426 Prefixes: ['CREL_', 'Above-']

# with num_crels fixed:
# 1 feats, new Best F1: 0.7481 Prefixes: ['CREL_']
# 2 feats, new Best F1: 0.7482 Prefixes: ['CREL_', 'num_crels']

1 feats, new Best F1: 0.7426 Prefixes: ['CREL_']
2 feats, new Best F1: 0.7468 Prefixes: ['CREL_', 'Prob-']
3 feats, new Best F1: 0.7472 Prefixes: ['CREL_', 'Prob-', 'CChain-']
4 feats, new Best F1: 0.7476 Prefixes: ['CREL_', 'Prob-', 'CChain-', 'CREL_Pair-']
No further improvement, stopping


In [ ]:
# run it against the full set of features for comparison
cv_filtered = []
for tr, test in cv_folds_mm:
    x_tr,x_test = filter_feats(tr, test, prefixes)
    cv_filtered.append((x_tr,x_test))

f1 = train_model_parallel(cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
                          pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
                          set_cr_tags=set_cr_tags, initial_weight=initial_weight)
f1

## Apply to Test Data

In [31]:
current_best

['CREL_', 'Prob-', 'CChain-', 'CREL_Pair-']

In [32]:
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

In [33]:
num_train = int(0.8 * len(xs_train_mm_fltr))
tmp_train_copy = list(xs_train_mm_fltr)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

## Determine the Optimal Number of Training Iterations

In [37]:
%%time
C = C
pa_type = 1
loss_type= "ml"
max_update_items = best_max_upd

mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=3, train_instance_fn = train_cost_sensitive_instance,
                                                        verbose=True,  early_stopping=False)

Epoch: 0 Train Accuracy: 0.7381 Test Accuracy: 0.7563
Epoch: 1 Train Accuracy: 0.7393 Test Accuracy: 0.7601
Epoch: 2 Train Accuracy: 0.7401 Test Accuracy: 0.7609
Epoch: 3 Train Accuracy: 0.7416 Test Accuracy: 0.7609
Epoch: 4 Train Accuracy: 0.7427 Test Accuracy: 0.7628
Epoch: 5 Train Accuracy: 0.7432 Test Accuracy: 0.7611
Epoch: 6 Train Accuracy: 0.7438 Test Accuracy: 0.7622
Epoch: 7 Train Accuracy: 0.7448 Test Accuracy: 0.7614
Epoch: 8 Train Accuracy: 0.7454 Test Accuracy: 0.7614
Epoch: 9 Train Accuracy: 0.7461 Test Accuracy: 0.7614
Epoch: 10 Train Accuracy: 0.7465 Test Accuracy: 0.7602
Epoch: 11 Train Accuracy: 0.7460 Test Accuracy: 0.7602
Epoch: 12 Train Accuracy: 0.7463 Test Accuracy: 0.7602
Epoch: 13 Train Accuracy: 0.7465 Test Accuracy: 0.7625
Epoch: 14 Train Accuracy: 0.7467 Test Accuracy: 0.7625
Epoch: 15 Train Accuracy: 0.7476 Test Accuracy: 0.7612
Epoch: 16 Train Accuracy: 0.7478 Test Accuracy: 0.7612
Epoch: 17 Train Accuracy: 0.7486 Test Accuracy: 0.7592
Epoch: 18 Train Accu

In [35]:
best_iterations

3

In [38]:
mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=initial_weight)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=best_iterations, early_stop_iters=best_iterations,
    train_instance_fn = train_cost_sensitive_instance, verbose=True, early_stopping=False)

# Best Test Acc: 0.7516

Epoch: 0 Train Accuracy: 0.7422 Test Accuracy: 0.7409
Epoch: 1 Train Accuracy: 0.7420 Test Accuracy: 0.7386
Epoch: 2 Train Accuracy: 0.7431 Test Accuracy: 0.7382
Epoch: 3 Train Accuracy: 0.7448 Test Accuracy: 0.7408
Epoch: 4 Train Accuracy: 0.7455 Test Accuracy: 0.7397
Epoch: 5 Train Accuracy: 0.7472 Test Accuracy: 0.7403
Epoch: 6 Train Accuracy: 0.7484 Test Accuracy: 0.7419
Epoch: 7 Train Accuracy: 0.7490 Test Accuracy: 0.7403
Epoch: 8 Train Accuracy: 0.7500 Test Accuracy: 0.7416
Epoch: 9 Train Accuracy: 0.7506 Test Accuracy: 0.7423
Epoch: 10 Train Accuracy: 0.7503 Test Accuracy: 0.7416
Epoch: 11 Train Accuracy: 0.7505 Test Accuracy: 0.7405
Epoch: 12 Train Accuracy: 0.7510 Test Accuracy: 0.7393
Epoch: 13 Train Accuracy: 0.7515 Test Accuracy: 0.7393
Epoch: 14 Train Accuracy: 0.7520 Test Accuracy: 0.7405
Epoch: 15 Train Accuracy: 0.7517 Test Accuracy: 0.7405
Epoch: 16 Train Accuracy: 0.7518 Test Accuracy: 0.7400
Epoch: 17 Train Accuracy: 0.7523 Test Accuracy: 0.7400
Epoch: 18 Train Accu

In [39]:
# try a smaller C
mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=initial_weight)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=5, early_stop_iters=5,
    train_instance_fn = train_cost_sensitive_instance, verbose=True, early_stopping=False)

# Best Test Acc: 0.7516

Epoch: 0 Train Accuracy: 0.7415 Test Accuracy: 0.7396
Epoch: 1 Train Accuracy: 0.7421 Test Accuracy: 0.7386
Epoch: 2 Train Accuracy: 0.7440 Test Accuracy: 0.7382
Epoch: 3 Train Accuracy: 0.7451 Test Accuracy: 0.7408
Epoch: 4 Train Accuracy: 0.7457 Test Accuracy: 0.7408
Best Test Acc: 0.7408


In [40]:
sorted(best_mdl.weights.items(), key = lambda tpl: -abs(tpl[1]))[0:50]

[('CREL_7:50', 0.1295000000000001),
 ('CREL_Causer:7->Result:50-MAX(prob)', 0.1261910460715546),
 ('CREL_Causer:7->Result:50-MIN(prob)', 0.11703694682578207),
 ('Prob-min-prob', 0.11678148330815484),
 ('Prob-prod-prob', 0.11534407092688548),
 ('Prob-5%-prob', 0.1132978689995122),
 ('Prob-10%-prob', 0.1098142546908705),
 ('CREL_3:50', 0.10900000000000008),
 ('CREL_1:50', 0.10750000000000008),
 ('CREL_Causer:1->Result:50-MIN(prob)', 0.10565276620728385),
 ('CREL_Causer:1->Result:50-MAX(prob)', 0.10562921121345513),
 ('CREL_Pair-Causer:1->Result:50|Causer:3->Result:50', 0.10250000000000008),
 ('CREL_6:7', 0.10150000000000008),
 ('CREL_Causer:1->Result:3-MAX(prob)', 0.09888870669879571),
 ('CREL_Causer:6->Result:7-pred-count=2', 0.09800000000000007),
 ('Prob-25%-prob', 0.09695825033207398),
 ('CREL_3:5', 0.09600000000000007),
 ('CREL_Causer:3->Result:50-MAX(prob)', 0.09503440904756608),
 ('CREL_Causer:3->Result:5-MAX(prob)', 0.09377645479089725),
 ('CREL_Causer:6->Result:7-MAX(prob)', 0.09

## TODO
- num-crels - add back in the logic to threshold these? But only if needed to improve results here (seemed to help essay parser)
- Re-run with more realistic initial hyper params
- Add in logic to store results to mongo
- We need to add hyper parameter tuning
- Do we want to just remove the BEAM search from this? It makes the explanation a lot more complex. But then again, it's the only way we can really add more crels that the model wouldn't otherwise parse
- Do we use the BEAM search with some de-duping? Although we already de-dupe to some extent anyways
